In [12]:
import pandas as pd 
import numpy as np
import requests
import json
import time
import datetime as dt
import re
import os.path
from openpyxl import load_workbook
import os

In [13]:
Area_Code = {
'012':['Algeria','DZA'],
'024':['Angola','AGO'],
'204':['Benin','BEN'],
'072':['Botswana','BWA'],
'854':['Burkina Faso','BFA'],
'108':['Burundi','BDI'],
'132':['Cabo Verde','CPV'],
'120':['Cameroon','CMR'],
'140':['Central African Republic','CAF'],
'148':['Chad','TCD'],
'174':['Comoros','COM'],
'178':['Congo','COG'],
'384':['Côte d’Ivoire','CIV'],
'180':['Democratic Republic of the Congo','COD'],
'262':['Djibouti','DJI'],
'818':['Egypt','EGY'],
'226':['Equatorial Guinea','GNQ'],
'232':['Eritrea','ERI'], #232
'748':['Eswatini','SWZ'],
'231':['Ethiopia','ETH'],
'266':['Gabon','GAB'],
'270':['Gambia','GMB'],
'288':['Ghana','GHA'],
'324':['Guinea','GIN'],
'624':['Guinea-Bissau','GNB'],
'404':['Kenya','KEN'],
'426':['Lesotho','LSO'],
'430':['Liberia','LBR'],
'434':['Libya','LBY'],
'450':['Madagascar','MDG'],
'454':['Malawi','MWI'],
'466':['Mali','MLI'],
'478':['Mauritania','MRT'],
'480':['Mauritius','MUS'],
'504':['Morocco','MAR'],
'508':['Mozambique','MOZ'],
'516':['Namibia','NAM'],
'562':['Niger','NER'],
'566':['Nigeria','NGA'],
'646':['Rwanda','RWA'],
'686':['Senegal','SEN'],
'690':['Seychelles','SYC'],
'694':['Sierra Leone','SLE'],
'706':['Somalia','SOM'],
'710':['South Africa','ZAF'],
'728':['South Sudan','SSD'],
'678':['So Tom & Prncipe','STP'],
'729':['Sudan','SDN'],
'768':['Togo','TGO'],
'788':['Tunisia ','TUN'],
'800':['Uganda','UGA'],
'834':['United Republic of Tanzania','TZA'], #835 834
'894':['Zambia','ZMB'],
'716':['Zimbabwe','ZWE'],
}

# Getting year for citation
D = dt.datetime.today()
y = D.year
area_code = [Area_Code[k][1] for k in Area_Code.keys()]

In [14]:
#Indicator ids that we want to download from IMF
IndicatorIDs = {'137506':'indx','146206':'NSO','68606':'indx'}

In [17]:
#Creating the dataFrame and organizaing the data
DataFrame = pd.DataFrame(columns = ['IndicatorName','IndicatorCode','AreaEnglish',\
                                    'AreaCode','Unitcode','Year','Value','DataSource',\
                                    'DataProviderId','SpecificSource'])
# Data Source
DataSource='United Nations Development Program'
DataProviderID=8

In [18]:
# Loop over the selected indicator ids and send request to get the json data
for ID in IndicatorIDs:
    print('Loading Data from: ',ID)
    url = f'http://ec2-54-174-131-205.compute-1.amazonaws.com/API/HDRO_API.php/indicator_id={ID}'
    response = None
    a = 0
    while response is None:
        try:
            a +=1
            print("atempt:",a)
            # make HTTP GET request to the target URL
            response = requests.get(url).json()
        except:
            time.sleep(3)
            continue    
    D = response['indicator_value']
    IndicatorCode = list(response['indicator_name'].keys())[0]
    IndicatorName = list(response['indicator_name'].values())[0]
    countries = response['country_name']
    # Filtering Data for African Countries
    dic = [{d: list(D[d].values())[0]} for d in D.keys() if d in area_code ]
    print(len(dic))
    for i in range(len(dic)):
        d = dic[i]
        code = list(d.keys())[0]
        country = countries[code]
        years = list(list(d.values())[0].keys())
        values = list(list(d.values())[0].values())
        indName = [IndicatorName]*len(years)
        indCode = [IndicatorCode]*len(years)
        cCode = [code]*len(years)
        cName = [country]*len(years)
        dSource = [DataSource]*len(years)
        dProviderId = [DataProviderID]*len(years)
        cite = f'=HYPERLINK("{url}")'
        SpecificSource = [cite]*len(years)
        unitcode = [IndicatorIDs[ID]]*len(years)
        
        D1 = np.array([indName,indCode,cName, cCode,unitcode,years,values,\
                       dSource,dProviderId,SpecificSource])

        df1 = pd.DataFrame(D1.T, columns=DataFrame.columns)

        #DataFrame = DataFrame.append(df1,ignore_index = True)
        DataFrame = pd.concat([DataFrame, df1], ignore_index=True, sort=False)
        
DataFrame.to_excel('HD.xlsx', index=False)
time.sleep(5)
print('All Done!')    

Loading Data from:  137506
atempt: 1
53
Loading Data from:  146206
atempt: 1
53
Loading Data from:  68606
atempt: 1
43
All Done!
